In [7]:
from data_utils import get_uci_data, common_processor_UCI, seed_all, california_housing_process, normalize, splitter
import numpy as np
from scipy.spatial import distance


import os
import yaml
from Experiments.EXP1.trainer import trainer, model_callByName, loss_callByName
from data_utils import get_uci_data, common_processor_UCI, seed_all, california_housing_process, normalize, splitter
from Experiments.EXP1.TestPerform import testPerform_muSigma, testPerform_kernel
import torch
import pandas as pd
from sklearn.ensemble import RandomForestRegressor




x, y = get_uci_data("concrete")

x_norm, x_normalizer = normalize(x)

x = x_norm

tr_idx, te_idx = splitter(int(len(x)/2), int(len(x)/2))

# we resample from tr_idx to create a shifted distribution

raw_train_X, raw_train_Y, test_X, test_Y = x[tr_idx], y[tr_idx], x[te_idx], y[te_idx]

x_bar = np.mean(raw_train_X, axis = 0)

cov_mat = np.cov(raw_train_X.T)

x_seed = np.random.multivariate_normal(mean = x_bar, cov = cov_mat, size = 1)[0]


resample_pickers = np.random.multivariate_normal(mean = x_seed, cov = 0.3 * cov_mat, size = 10000)

picked_idx = []

for picker in resample_pickers:
    
    closest_index = distance.cdist(raw_train_X, [picker]).argmin()
    picked_idx.append(closest_index)
    
train_X = raw_train_X[np.array(picked_idx)]
train_Y = raw_train_Y[np.array(picked_idx)]

train_X, test_X = torch.Tensor(x), torch.Tensor(test_X)
train_Y, test_Y = torch.Tensor(y).to(torch.device("cuda")), torch.Tensor(test_Y).to(torch.device("cuda"))

# benchmarks

In [8]:



for modelname in [ "HNN", "MC_drop", "DeepEnsemble"]:

    # train base model
    print("model: "+ modelname +" on data: shifted concrete")

    with open(os.getcwd()+"/Experiments/EXP1/config_bin/"+modelname+"_on_wine_config.yml", 'r') as file:
        base_configs = yaml.safe_load(file)

    base_misc_info = base_configs["misc_info"]
    base_train_config= base_configs["training_config"]

    base_misc_info["model_config"]["hidden_layers"] = [10, 5]
    base_misc_info["model_config"]["n_input"] = x.shape[1]

    base_train_config["LR"] = 5E-3
    base_train_config["bat_size"] = 64

    crits_dic = {}

    

    base_model = model_callByName[base_misc_info["model_init"]](**base_misc_info["model_config"])

    

    trainer(
        raw_train_X = train_X,
        raw_train_Y = train_Y,
        model = base_model,
        training_config = base_train_config,
        harvestor = None,          
        misc_info = base_misc_info,
        diff_trainingset = True
    )



    record = testPerform_muSigma(test_X, test_Y, model_name= modelname, model = base_model)

    print(record)

model: HNN on data: shifted concrete
{'rmse_loss': 12.653321266174316, 'mean_std_norm_loss': 2.8693671226501465, 'MACE_muSigma': 0.02073409967124462, 'AGCE_muSigma': 0.0565212145447731, 'CheckScore_muSigma': 3.3146910667419434}
model: MC_drop on data: shifted concrete
{'rmse_loss': 11.748173713684082, 'mean_std_norm_loss': 3.0167088508605957, 'MACE_muSigma': 0.06425502896308899, 'AGCE_muSigma': 0.10216015577316284, 'CheckScore_muSigma': 3.407999277114868}
model: DeepEnsemble on data: shifted concrete
{'rmse_loss': 12.2550630569458, 'mean_std_norm_loss': 2.8191704750061035, 'MACE_muSigma': 0.022408394142985344, 'AGCE_muSigma': 0.06186332553625107, 'CheckScore_muSigma': 3.1794943809509277}


# VanillaPred

In [9]:
modelname =  "vanillaPred"

# train base model
print("model: "+ modelname +" on data: shifted concrete")

with open(os.getcwd()+"/Experiments/EXP1/config_bin/"+modelname+"_on_wine_config.yml", 'r') as file:
    base_configs = yaml.safe_load(file)

base_misc_info = base_configs["misc_info"]
base_train_config= base_configs["training_config"]

base_misc_info["model_config"]["hidden_layers"] = [10, 5]
base_misc_info["model_config"]["n_input"] = x.shape[1]

base_train_config["LR"] = 5E-3
base_train_config["bat_size"] = 64

crits_dic = {}



pred_model = model_callByName[base_misc_info["model_init"]](**base_misc_info["model_config"])

N_train = int(len(train_X)*0.7)
N_recal = int(len(train_X)*0.3)

tr_idx, recal_idx = splitter(N_train, N_recal)



trainer(
    raw_train_X = train_X[tr_idx],
    raw_train_Y = train_Y[tr_idx],
    model = pred_model,
    training_config = base_train_config,
    harvestor = None,          
    misc_info = base_misc_info,
    diff_trainingset = True
)

model: vanillaPred on data: shifted concrete


In [11]:
width = 10

record = testPerform_kernel(
        test_X, test_Y, train_X[recal_idx], train_Y[recal_idx], 
        model_name = "vanillaKernel", model= pred_model, wid = width)
print(record)               

{'MACE_Loss': 0.013928954489529133, 'AGCE_Loss': 0.06772049516439438, 'CheckScore': 2.463222026824951}
